In [1]:
import csv

import pandas as pd
from pandas import DataFrame

import numpy as np
from numpy.random import randn

import tensorflow as tf
from numpy import arange,array,ones

import tensorflow_hub as hub
import os

os.environ["TFHUB_CACHE_DIR"] = str(os.getcwd() + '/tfhub')

W0527 22:20:53.778329 17924 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [2]:
## open data file
with open('train.csv', 'r') as csvfile:
    trainCSV = csv.reader(csvfile, delimiter = ',')
    
train_raw = pd.read_csv("train.csv", error_bad_lines=False, header = None, delimiter = ',')

train_op = train_raw
train_op.head()

C:\Users\Sirui\Anaconda3\envs\datares\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,33,34,35,37,38,39,40,41,42,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
0,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
1,59848,0.0,"This is so cool. It's like, 'would you want yo...",0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
2,59849,0.0,Thank you!! This would make my life a lot less...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
3,59852,0.0,This is such an urgent design problem; kudos t...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
4,59855,0.0,Is this something I'll be able to install on m...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4


In [3]:
## Data cleaning >> select relevant columns
train_text = train_op[[2]]
train_text = train_text.drop([0])
train_text_list = train_text.values.tolist()
text_array = []
for i in train_text_list:
    j = str(i)
    j = j[2:len(j)-2]
    text_array.append(j)
 

In [4]:
text_array


["This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!",
 "Thank you!! This would make my life a lot less anxiety-inducing. Keep it up, and don't let anyone get in your way!",
 'This is such an urgent design problem; kudos to you for taking it on. Very impressive!',
 "Is this something I'll be able to install on my site? When will you be releasing it?",
 'haha you guys are a bunch of losers.',
 'ur a sh*tty comment.',
 'hahahahahahahahhha suck it.',
 'FFFFUUUUUUUUUUUUUUU',
 'The ranchers seem motivated by mostly by greed; no one should have the right to allow their animals destroy public land.',
 "It was a great show. Not a combo I'd of expected to be good together but it was.",
 'Wow, that sounds great.',
 'This is a great story. Man. I wonder if the person who yelled "shut the fuck up!" at him ever heard it.',
 'This seems like a step in the right direction.',
 'It\\\'s ridiculous that these guys are being called "protesters". Being ar

In [4]:
## data cleaning >> change data type for further modeling 
train_target = train_op[[1]]
train_target = train_target.drop([0])
list1 = train_target.values.tolist()
type(list1)
target_array = np.reshape(list1, (len(list1),))
#for item in list1:
 #   string = item[2:len(item)-2]
target_array = target_array.astype(np.float)

In [5]:
sample_outputt = target_array[0:1000]


In [6]:
## process texts into matrix
x = tf.placeholder(dtype=tf.string, shape=[None])
embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-large/3")
embeddings = embed(x)

Instructions for updating:
Colocations handled automatically by placer.


W0527 22:35:21.774483 17924 deprecation.py:323] From C:\Users\Sirui\Anaconda3\envs\datares\lib\site-packages\tensorflow\python\ops\control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0527 22:35:24.341129 17924 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [7]:
## text embeddings
with tf.Session() as sess:
    sess.run([tf.global_variables_initializer(), tf.tables_initializer()])
    output = sess.run(embeddings, feed_dict = {
        x: ["I hate brocolli.", "Avengers Endgame is an overrated movie."]
    })

In [8]:
print(output)

[[-0.01669056 -0.06686931 -0.00095109 ...  0.05440774  0.04717208
   0.01076814]
 [ 0.02601201  0.05873855  0.0732425  ...  0.03494182  0.06604067
  -0.02768116]]


In [9]:
output.shape

(2, 512)

In [20]:
sample_text = text_array[0:1000]
sample_target = target_array[0:1000]

In [11]:
len(sample_target)

100

In [12]:
sample_output = sample_target

In [13]:
embeddings

<tf.Tensor 'module_apply_default/Encoder_en/hidden_layers/l2_normalize:0' shape=(?, 512) dtype=float32>

In [14]:
# forward propogration
hidden = tf.layers.dense(inputs = embeddings, units = 1024, activation = tf.nn.relu)

output = tf.layers.dense(inputs = hidden, units = 1, activation = tf.nn.sigmoid)

Instructions for updating:
Use keras.layers.dense instead.


W0527 22:37:47.792460 17924 deprecation.py:323] From <ipython-input-14-a51f6df42faf>:2: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.


In [21]:
# backforward propogation
y = tf.placeholder(dtype=tf.float32, shape=[None])
output1 = tf.reshape(output, [100,])

cost = tf.losses.mean_squared_error(y, output1) 
optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

In [22]:
# divide into different sets for easier operations
with tf.Session() as sess:
    sess.run([tf.global_variables_initializer(), tf.tables_initializer()])  
    
    for i in range(0,5):
        for j in range (0, 10):
            
            loss, last = sess.run([cost, optimizer], feed_dict = {
                 x: sample_text[j*100: 100*(j+1)],
                 y: sample_target[j*100: 100*(j+1)]
            })
        print (loss)


0.20663445
0.19848442
0.19072333
0.18333802
0.17630798
